In [1]:
import random
import torch
import torch.optim as optim
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from torch.autograd import Variable
from shared import (squadIteratorGraph, readDicFromFile, loadSparseM, 
                    initEdgeVectors, mkGraph, nearestNodes, processOneExample, 
                    readFrequencyFile,
                    getDistanceMatrix,
                    spanDistance)

In [2]:
class MyLayer(torch.nn.Module):
    def __init__(self, F_size):

        super(MyLayer, self).__init__()
        #self.L = torch.nn.Parameter(torch.randn(F_size, F_size))
        self.L = torch.nn.Linear(F_size, 1)
        #self.p_q = torch.nn.Parameter(torch.randn(1))
        #self.p_s = torch.nn.Parameter(torch.randn(1))

        #self.adjencyMatrixQ = adjencyMatrixQ
        #self.adjencyMatrixS = adjencyMatrixS

    def forward(self, X):
        #print(self.adjencyMatrixQ.size())
        #print("X", X)
        I = Variable(torch.eye(self.adjencyMatrixQ.size()[0]))
        #print(self.adjencyMatrixQ.squeeze_)
        #print(self.adjencyMatrixQ.matmul(X))
        #W_q = (self.adjencyMatrixQ + I) * self.p_q.expand_as(self.adjencyMatrixQ)
        #W_s = (self.adjencyMatrixS + I) * self.p_s.expand_as(self.adjencyMatrixS)
        #W = W_q + W_s
        #return W.matmul(X).matmul(self.L).sigmoid()
        return self.L((self.adjencyMatrixQ + I).matmul(X))
        

    def bindAdjencyMatrix(self, adjencyMatrixQ):
        self.adjencyMatrixQ = adjencyMatrixQ


In [2]:
#------- train ------
vocab = readDicFromFile('squad_vocab_100K')
matrix = loadSparseM('matrix_100K_reduced')
trainIter = squadIteratorGraph('train-v1.1.json', vocab, matrix)

model = MyLayer(4)

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)



for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, ((gr, idxs), answer) in enumerate(trainIter, 0):
        if (i < 118): continue
        #print(answer)
        # get the inputs
        q_idxs = [v["idx"] for v in idxs if v["in_question"]]
        #print(q_idxs)
        #print(len(idxs), gr.shape[0])
        p_idxs = [v["idx"] for v in idxs if not v["in_question"]]
        inputs = initEdgeVectors(gr, q_idxs)
        #print("inputs", inputs.astype(int).dtype)
        inputs = torch.from_numpy(inputs).float()
        #print(type(inputs))
        labels = [1.0 if v["in_answer"] else 0 for v in idxs]
        labels = torch.FloatTensor(labels).view(-1, 1)
        
        adjency_for_q = gr.copy().astype(float)
        #print("adjency_for_q", adjency_for_q.dtype)
        
        adjency_for_q[p_idxs, :] = 0
        adjency_for_q[:, p_idxs] = 0
        #print("gr copy", adjency_for_q)
        #print("adjency_for_q squeezed", [v for v in adjency_for_q.getA1() if v > 0])
        #print("adjency_for_q - 1", [v for v in adjency_for_q.getA1() if v > 0])
        #print(type(adjency_for_q))
        #print("adjency_for_q", torch.from_numpy(np.asarray(adjency_for_q)))
        #print("type adjency_for_q", adjency_for_q.dtype, type(adjency_for_q[0,0]))
        #print(type([v for v in adjency_for_q.getA1() if v > 0]))
        #print(type([1,2,3]))
        #t = torch.from_numpy(np.array([v for v in adjency_for_q.getA1() if v > 0]))
        #t.type(torch.IntTensor)
        #print('adjency_for_q type', torch.from_numpy(adjency_for_q.getA()).type())
        adjency_for_q = Variable(torch.from_numpy(adjency_for_q.getA()).float(), requires_grad = False) 
        #print("adjency_for_q - 2", adjency_for_q.data.numpy())
        model.bindAdjencyMatrix(adjency_for_q)
        #print("inputs type", type(inputs))
        #print("labels type", type(labels))


        # wrap them in Variable
        inputs, labels = Variable(inputs, requires_grad = False), Variable(labels, requires_grad = False)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        #print(outputs)
        #print(labels)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        #print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, loss.data[0]))
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

TypeError: squadIteratorGraph() missing 1 required positional argument: 'freq_dict'

In [10]:
m1 = Variable(torch.FloatTensor([[0, 1, 0], [1, 0, 1], [0, 1, 0]]), requires_grad = False)
m2 = Variable(torch.FloatTensor([[0, 0, 1], [0, 0, 0], [1, 0, 0]]), requires_grad = False)
X = Variable(torch.randn(3, 4), requires_grad = False) 
model = MyLayer(4)
model.bindAdjencyMatrix(m1)
X1 = model(X)

In [2]:
vocab = readDicFromFile('vocab500K')
matrix = loadSparseM('matrix_500K_sc4_reduced')
freq_dict = readFrequencyFile("freq_vocab500K")
trainIter = squadIteratorGraph('train-v1.1.json', vocab, matrix, freq_dict)


In [3]:
for i in range(20):
    next(trainIter)
    


In [14]:
(gr, idxs), answer, question, paragraph = next(trainIter)
print("P: ", paragraph)
print("Q: ", question)
print("A: ", answer)

P:  Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Q:  What is in front of the Notre Dame Main Building?
A:  a copper statue of Christ


In [14]:
q_idxs = [v["idx"] for v in idxs if v["in_question"]]
q_idxs_words = [(v["idx"], v['word']) for v in idxs if v["in_question"]]
print("Q idxs:", q_idxs)
print("Q:", q_idxs_words)
a_idxs = [i for i, v in enumerate(idxs) if v["in_answer"]]
print("A idxs:", a_idxs)
all_idxs = [v["idx"] for v in idxs]
dic_idx_to_word = dict([(v["idx"], v["word"]) for v in idxs])
#idxs

Q idxs: [1, 4, 10, 13, 14, 15, 18, 68, 69, 70, 71]
Q: [(1, 'notre'), (4, 'students'), (10, 'at'), (13, 'in'), (14, 'first'), (15, 'year'), (18, 'was'), (68, 'what'), (69, 'created'), (70, 'dame'), (71, 'assist')]
A idxs: [14, 15, 16, 17]


In [8]:
gr[q_idxs, :][:, a_idxs]

matrix([[13543, 56990],
        [ 3451,  3071],
        [  259,  3696],
        [  423,  3816],
        [ 4788,  2047],
        [ 2280,   840],
        [33713, 53376]], dtype=uint16)

In [8]:
graph = mkGraph(gr)
nearest = nearestNodes(graph, q_idxs, all_idxs)
[(idx, dic_idx_to_word[idx], dist, freq_dict[dic_idx_to_word[idx]]) for (idx, dist) in nearest]

[(28, 'major', 17804.846153846152, '1036686'),
 (10, 'at', 17807.23076923077, '13209400'),
 (72, 'for', 18034.23076923077, '20217315'),
 (13, 'in', 18305.384615384617, '66492400'),
 (23, 'their', 18607.923076923078, '4503807'),
 (15, 'year', 18627.538461538461, '2845442'),
 (17, 'program', 18736.0, '655622'),
 (62, 'by', 18749.153846153848, '17444726'),
 (11, 'school', 18792.538461538461, '2950021'),
 (31, 'is', 18810.307692307691, '21279224'),
 (14, 'first', 19318.384615384617, '5591677'),
 (24, 'before', 19349.307692307691, '1636083'),
 (64, 'news', 19538.23076923077, '492610'),
 (71, 'with', 19673.384615384617, '16941836'),
 (18, 'was', 20221.384615384617, '26107810'),
 (44, 'which', 20303.461538461539, '6588422'),
 (29, 'each', 20304.461538461539, '1190204'),
 (41, 'give', 20421.384615384617, '268419'),
 (73, 'new', 20959.923076923078, '5269523'),
 (4, 'students', 21183.538461538461, '669686'),
 (0, 'all', 21333.538461538461, '3448959'),
 (35, 'from', 21717.153846153848, '12751477'

In [50]:
nearestNodes(graph, [13], [12])

[(12, 37842.0)]

In [15]:
#question = 'undergraduate'
# paragraph = "All of Notre Dame's undergraduate students are a part of one of the five undergraduate colleges at the school or are in the First Year of Studies program."
# #question = 'tissue'
# #answer = ''
# (gr, idxs), answer, question, paragraph = processOneExample(vocab, matrix, answer, question, paragraph, freq_dict)
# print("P: ", paragraph)
# print("Q: ", question)
# print("A: ", answer)
# q_idxs = [v["idx"] for v in idxs if v["in_question"]]
# all_idxs = [v["idx"] for v in idxs]
# dic_idx_to_word = dict([(v["idx"], v["word"]) for v in idxs])
# graph = mkGraph(gr)
# nearest = nearestNodes(graph, q_idxs, all_idxs, avg=False)


# new approach
#span = '''At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.''' # list of words (paragraph substring)
distanceMatrix, dm_word_idx_map = getDistanceMatrix(matrix, paragraph, question, vocab)

for idx, sentence in enumerate(paragraph.split('.')):
    d = spanDistance(sentence, question, distanceMatrix, dm_word_idx_map)
    print(f'sentence #{idx} has distance {d}')
    #print(f"foo is {bar}.")
#[(idx, dic_idx_to_word[idx], dist, freq_dict[dic_idx_to_word[idx]]) for (idx, dist) in nearest]


sentence #0 has distance 0.04923540531571137
skip unknown word: 's
sentence #1 has distance 0.0632554063281528
sentence #2 has distance 0.04347914061466824
sentence #3 has distance 0.008773189608067146
sentence #4 has distance 0.0764841085694712
sentence #5 has distance 0.04605734265171164
sentence #6 has distance 0.04147372590427224


ZeroDivisionError: float division by zero

In [6]:
dm_word_idx_map

{'<NNP>': 6,
 '<NUM>': 38,
 'allegedly': 53,
 'and': 16,
 'appear': 54,
 'appeared': 37,
 'architecturally': 0,
 'arms': 21,
 'at': 34,
 'atop': 5,
 'basilica': 27,
 'behind': 26,
 'catholic': 3,
 'character': 4,
 'connects': 45,
 'copper': 19,
 'did': 52,
 'direct': 42,
 'dome': 8,
 'drive': 41,
 'end': 39,
 'facing': 17,
 'front': 15,
 'gold': 7,
 'golden': 10,
 'grotto': 33,
 'has': 2,
 'immediately': 13,
 'in': 14,
 'is': 9,
 'it': 18,
 'legend': 23,
 'line': 43,
 'main': 40,
 'marian': 28,
 'modern': 49,
 'next': 24,
 'of': 12,
 'place': 29,
 'prayer': 30,
 'reflection': 31,
 'replica': 32,
 'reputedly': 36,
 'school': 1,
 'simple': 48,
 'statue': 11,
 'statues': 47,
 'stone': 50,
 'that': 44,
 'through': 46,
 'to': 25,
 'upraised': 22,
 'where': 35,
 'whom': 51,
 'with': 20}

In [16]:
for idx, s in enumerate(paragraph.split('.')):
    print(f'{idx} {s} \n')

0 Architecturally, the school has a Catholic character 

1  Atop the Main Building's gold dome is a golden statue of the Virgin Mary 

2  Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes" 

3  Next to the Main Building is the Basilica of the Sacred Heart 

4  Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection 

5  It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858 

6  At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary 

7  



In [17]:
question

'What is in front of the Notre Dame Main Building?'

In [18]:
distanceMatrix

matrix([[ 0.        ,  0.15056995,  0.15819735, ...,  0.12762819,
          0.09158271,  0.14835831],
        [ 0.15056995,  0.        ,  0.03894468, ...,  0.05883895,
          0.09925219,  0.05203693],
        [ 0.15819735,  0.03894468,  0.        , ...,  0.05822956,
          0.10525457,  0.05075235],
        ..., 
        [ 0.12762819,  0.05883895,  0.05822956, ...,  0.        ,
          0.06984629,  0.05392935],
        [ 0.09158271,  0.09925219,  0.10525457, ...,  0.06984629,
          0.        ,  0.09147715],
        [ 0.14835831,  0.05203693,  0.05075235, ...,  0.05392935,
          0.09147715,  0.        ]])

In [12]:
distanceMatrix.shape

(48, 48)